# Lesson S2 - Mean Areal Calculation

## Overview

Often times, the mean areal quantities are desired to get a better sense of overall impact of an event in an area. Here we have compiled a set of functions and scripts which help you to calculate the Mean Areal values from any gridded data (the focus here is precipitation, but could be also used for snow or other components of the model). Here we provide two ways to calculate the mean areal precipitation. The first method is suitable for the small areas and few basins, the second is how we calculate the mean areal precipitaiton at the CONUS level which is much faster than the first method. 

## First method for mean areal calculation (using a spatial mask)
The procudure for caluclating the mean areal precipitation using this method is as follows:

1. Trace out the routelink file and find all the NHD catchments that are draining to the outlet/gage location
2. Next, read the spatial weight file and create a mask 
3. Read the precipitaion 2D field (or any other), use the mask and calculate the mean area field

Let's read the Route_Link.nc file and find out the linkId of the location we are interested in

In [ ]:
rtlinkFile <- "~/nwm-training/example_case/NWM/DOMAIN/Route_Link.nc"
rl <- rwrfhydro::ReadRouteLink(rtlinkFile)  
linkId <- subset(rl$link, rl$site_no == "01447720")    
print(linkId)

The outlet gage for our case study is located on the NHD reach with the ComId of 4185779. Now that we have the comId of the gage location, we will trace up and find all the reaches that drain to this link. Here, we provided a simple function `traceUp` that will find all the uplinks that drain to the ComId of 4185779. Note this process will be slower for larger domains. 

In [ ]:
# Function to Trace upstream in route link                                                                      
traceUp <- function(linkid, rl) {                                                                   
    uplinks <- c(linkid)                                                                            
    donelinks <- c()                                                                                
    while (length(uplinks) > 0) {                                                                   
        i <- uplinks[1]                                                                             
        donelinks <- c(donelinks, i)                                                                
        newlinks <- subset(rl$link, rl$to==i)                                                       
        uplinks <- c(uplinks, newlinks)                                                             
        uplinks <- unique(subset(uplinks, !(uplinks %in% donelinks)))                               
    }                                                                                               
    donelinks                                                                                       
}   

# Calling Trace up function
uplinks <- traceUp(linkId, rl) 

# print the uplinks
print(uplinks)

As shown here, there are 194 reaches that are draining to the outlet location. The next step is to read in the spatial weight file and find out which pixels are contributing to the outlet location. **NOTE** we are using the spatial weight file that is been provided for this domain and it is on the fine resolution (250 m). 

In [ ]:
# Read the spatial weight file
spwtFile <-  "~/nwm-training/example_case/NWM/DOMAIN/spatialweights.nc"
spwtList <- rwrfhydro::ReadWtFile(spwtFile) 

# subset the spatial weight file
spwt.data <- spwtList[[1]]     
spwt.data <- subset(spwt.data, spwt.data$IDmask %in% uplinks)  
print(head(spwtList))


The above list shows all the basins (IDmask is the comId of the NHD catchment) that drains to the outlet locations with percentage of each pixel (regridweight) that contributes to the given catchment. The next step is to create mask files for both the routing and LSM grid that will be used for mean areal calculations. Let's first find the total percentage of each cell that falls into the contributing area of the outlet location. 

In [ ]:
# Calculate what percentage of each cell contributes to the desired location
library(plyr)
spwt.data.sumcell <- ddply(spwt.data, .(i_index, j_index), summarize, sumwt=sum(regridweight))
print(spwt.data.sumcell)

Next we create the mask for the finer resolution grid (hydro grid). We get the size of the mask from the Fulldom_hires.nc file, then create an empty 2D mask, and fill it with the percentage of each cell that falls into the contributing area of the outlet location. 

In [ ]:
# Define a 2D field at the hydro resolution

# Get the hydro dimensions from thr Fulldom file
fulldomFile <- "~/nwm-training/example_case/NWM/DOMAIN/Fulldom_hires.nc"
ncid <- ncdf4::nc_open(fulldomFile)
xlen.hyd <- ncid$dim$x$len
ylen.hyd <- ncid$dim$y$len
ncdf4::nc_close(ncid)

# print the number of pixels
print(paste0("Number of pixels in west-east: ", xlen.hyd))
print(paste0("Number of pixels in south-north : ", ylen.hyd))

# create an empty 2D mask using the dimension of the hydro grid
mskvar.hyd <- matrix(0, nrow=xlen.hyd, ncol=ylen.hyd) 

# for each pixel assign what percentage of it falls in the contributing area of the gage/outlet
for (n in 1:nrow(spwt.data.sumcell)) {                                                        
    mskvar.hyd[spwt.data.sumcell$i_index[n], spwt.data.sumcell$j_index[n]] <- spwt.data.sumcell$sumwt[n]
} 

# display the mask
image(mskvar.hyd)

The above image shows the mask that can be used for mean areal calculation of the variables on the hydro grid. However, precipitation is on the LSM grid, and therefore we need to create another mask with the LSM resolution. We get the dimension of the LSM grid from the geogrid file, and resample the hydro mask to LSM mask. 

In [ ]:
# Create 2D field at the LSM resolution
library(raster)

# Get the LSM dimension from the geogrid file
geoFile <- "~/nwm-training/example_case/NWM/DOMAIN/GEOGRID_LDASOUT_Spatial_Metadata.nc"
ncid <- ncdf4::nc_open(geoFile)
xlen.geo <- ncid$dim$x$len
ylen.geo <- ncid$dim$y$len
ncdf4::nc_close(ncid)

# print the dimension of the LSM grid 
print(paste0("Number of pixels in west-east: ", xlen.geo))
print(paste0("Number of pixels in south-north : ", ylen.geo))

# create an empty 2D mask using the dimension of the LSM grid
mskvar.lsm <- matrix(1, xlen.geo, ylen.geo)

# resample the hydro mask to LSM mask
mskvar.lsm <- as.matrix(resample(raster(mskvar.hyd), raster(mskvar.lsm), 'bilinear'))

# display the LSM mask
image(mskvar.lsm)

Now that we have the mask, let's read a sample precipitaiton file and calculate the MAP. 

In [ ]:
# Defien a function to calculate basin mean 
basin_avg <- function(myvar, mymsk, minValid=-9998) {
    myvar[which(myvar<minValid)]<-NA
    mymsk[which(is.na(myvar))]<-NA
    sum(mymsk*myvar, na.rm=TRUE)/sum(mymsk, na.rm=TRUE)
}  

# address to one of the precipitation files
inputFile = "~/nwm-training/example_case/FORCING/2011082804.LDASIN_DOMAIN1"

# using ncdump function in rwrfhydro to read the RAINRATE variable 
rain <- rwrfhydro::ncdump(file = inputFile, variable = "RAINRATE", quiet = TRUE) 

# convert it mm/s to mm/hr
rain <- rain * 3600 

#calculate the Mean Areal Precipitation and print the value
MAP <- basin_avg(rain, mskvar.lsm)
print(MAP)

## Second method for mean areal calculation (Intervening area)

Although the example case here is pretty small and this calculation may take a few seconds no matter what approach you may take, the main thinking behind the second approach is to apply this at the CONUS scale and therefore it should be computationally cheap. There are 7541 USGS gages in the routelink file and the goal is to calculate mean areal precipitation over the contributing area of each gage. To do that we take three steps:

1. Calculate precipitation over each NHD catchment 
2. Calculate precipitation over intervening areas 
3. Calculate precipitation over contributing area

We will attempt to explain the procedure of the above steps using a much simpler test case and then we will apply it to the training example case. Lets consider the following domain which has 2 USGS gages and 6 NHD Cathments as our sample. Our ultimate goal is to calculate the mean areal precipitation that is contributing to the two USGS gage locations. 

<img src="lesson_s2_imp_R/MAP_fig1.png" width="600">

### Calculate the mean areal over each NHD catchment

As you know by now NWM makes use of the NHD Catchments, and the information about the overlap of the terrain routing grids (finer grid) are stored in the spatial weight file. There are two spatial weight files available tagged as 1 km and 250 meter, used for the Long Range and Full Physics configuration, respectively. First we find out what pixels are contributing to each NHD catchments by subsetting the spatial weight file. For example, for the catchment 9202601, there are 15 pixels contributing to it. 

<img src="lesson_s2_imp_R/MAP_fig2.png" width="400">

We need the following fields from the spatialweight file (note I am showing only the first three entries):

| IDmask | i_index | j_index | regridweight |
| ------------- | ------------- | ------------- | ------------- |
| 9202601 |   3744  |  1553  | 0.33147130 |
| 9202601 |  3744   | 1554 |  0.89377742 |
| 9202601 |   3744  |  1555  | 0.52243955 |

And you calcualate the mean precip over this NHD catchment as follows:
$$MAP_{9202601} = \frac{\sum\limits^{13}_{i=1}P_i*regridweight_i)}{\sum\limits^{13}_{i=1}regridweight_i}$$

Note this step can be ommitted, and you can directly calculate the precip over the intervening areas. 

### Calculate precipitation over intervening areas 

To be computationally efficient, we define the itervening areas. The intervening area for a given gage is all the NHD catchment between the desired gage and the immediate upstream gages. For example the white area in the following figure shows the intervening area to gage 0212427947 which is the same as its contributing area since there is no upstream gage, and the blue catchments are the intervening area to gage 0212430293. So for each gage we will calculate the mean areal precip over the intervening areas. 

![MAP_fig3.png](lesson_s2_imp_R/MAP_fig3.png)

### Calculate precipitation over contributing area
    
And finally weight avergae all the intervening areas that drain to your desired location to get the mean areal precipitation over the contributing area. 

$$MAP_{0212430293} = \frac{(MAPint_{0212427947}*AREAint_{0212427947} + MAPint_{0212430293}*AREAint_{0212430293})}{AREAint_{0212427947}+AREAint_{0212430293}}$$

### Perform the mean areal calculation on the example case

Now let's do this calculation on this training example case. Since the function that is required is quite lenghty, we have stored the functions in a separate Rscript that is supplemented with this lesson. We first source that script. 

In [ ]:
# load the required functions for the rest of the lessons. 
# This script has 4 functions that we explain later, 
# GetUpStreamGages, GetInterveningNHD, CalcMeanAreal, CalcMeanArealCont

source("~/nwm-training/lessons/lesson_s2_imp_R/Lesson-S2-Calc_Mean_Areal.R")

First step is to calcualte the mean areal precipitation is calculated over each NHD catchment (there are about 2.7 million of those catchments in US). To do that, you need the sptial weight file. 


In [ ]:
# address to one of the precipitation files
inputFile = "~/nwm-training/example_case/FORCING/2011082804.LDASIN_DOMAIN1"

# using ncdump function in rwrfhydro to read a RAINRATE variables 
rain <- rwrfhydro::ncdump(file = inputFile, variable = "RAINRATE", quiet = TRUE) 

# convert it mm/s to mm/hr
rain <- rain * 3600 

# address to the spatial weight file
spatialWeightFile <-  "~/nwm-training/example_case/NWM/DOMAIN/spatialweights.nc"

# calculate the precip over each NHD catchment and print the values
meanP <- CalcMeanAreal(spatialWeightFile, rain)
print(meanP)

So there were 374 NHD catchments in the rectangle domain. Next is to calculate the mean areal precip over each intervening area, and therefore we need to find the information about all the upstream USGS gages above each gage. And that can be done using the function `GetUpStreamGages` in the Rscript we sourced earlier. This function is using the reindexing files that we also mentioned in the subsetting lesson. 

In [ ]:
# Find out all the upstream gages
routelinkFile <- "~/nwm-training/example_case/NWM/DOMAIN/Route_Link.nc"
upStreamGages <- GetUpStreamGages(routelinkFile)

print(upStreamGages)

Note that it first created the reindexing files, using the rwrfhdyro function `ReExpressRouteLink` and then listed all the gages and their upstream gages. Note creation of the reindexing file takes several hours for the CONUS. In the sample domain, there are 2 gages. `01447680` has no gage upstream of it, and gage `01447720` has one gage upstream of it that is `01447680`. 

Next step is to find all the intervening NHD catchments for these two gages. You could use the function `GetInterveningNHD` for this task. 

In [ ]:
# List all the intervening NHD catchments (catchments between a gage and its immediate upstream gages)
interveningNHD <- GetInterveningNHD(routelinkFile, spatialWeightFile)

print(interveningNHD)

`GetInterveningNHD` return a data.table having the information on each gage and its corresponding comIds (within the intervening area of the gage) as well as the area of the catchment (**WATCH!** this is not actual area, this is fraction of pixels within a given NHDPlus catchment, for example for the case of NWM, 16 would be 1 sqkm since the routing grid are 250meter and the spatial weight file used here is for the overland flow routing grid (Fulldom grid))

Final step is to calculate the mean areal precipitation for the intervening and contributing area, and both steps are done in one function call. 

In [ ]:
# This function asks for the mean areal values over the NHD catchments (from step1), 
#    the intervening NHD catchment information and upstream gage information 

contP <- CalcMeanArealCont(meanArealDT = meanP, interveningNHD, upStreamGages)

print(contP)

The above tells the mean areal precipitation over the intervening area (`meanIntervening`) and contributing area (`meanContributing`) for both gages. For gage `01447680` since there was no gage above it, both the intervening and contributing MAPs are the same. For gage `01447720` which is the outlet location of our test case, the contributing MAP is 8.47 which is similar to what we got from the first MAP calculation method.

© UCAR 2019